# Day 35

Today I'm going to pull ADP data from as many seasons as possible. I'll utilize the public API from [Fantasy Football Calculator](https://help.fantasyfootballcalculator.com/article/42-adp-rest-api).  

Eventually I want to see how a player's fantasy football production compares to their ADP. Is it the case that the top drafted players actually end up performing the best? How does this comparison hold up historically?

In [30]:
import pandas as pd
import numpy as np
import requests
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

In [25]:
def fantasy_calculator_adp(scoring='ppr', teams=12, season=2022):
    
    try:
        base_url = 'https://fantasyfootballcalculator.com'
        
        response = requests.get(f"{base_url}/api/v1/adp/{scoring}?teams={teams}&year={season}")
        
        df = pd.DataFrame(response.json()['players'])

        # Add identification columns
        df['season'] = season
        df['teams'] = teams
        df['scoring'] = scoring

        return df
    
    except:
        print(f"No data available for combination: {scoring}, {teams}, {season}")

In [27]:
df = fantasy_calculator_adp(scoring='ppr', teams=12, season=2021)
df.head()

,player_id,name,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye,season,teams,scoring
0,2434,Christian McCaffrey,RB,CAR,1.2,1.01,1368,1,5,0.5,13,2021,12,ppr
1,2432,Dalvin Cook,RB,MIN,2.5,1.03,1292,1,6,0.7,7,2021,12,ppr
2,2439,Alvin Kamara,RB,NO,3.8,1.04,1500,1,8,1.0,14,2021,12,ppr
3,2350,Derrick Henry,RB,TEN,4.0,1.04,460,1,8,1.3,6,2021,12,ppr
4,2343,Ezekiel Elliott,RB,DAL,5.3,1.05,1148,2,12,1.1,9,2021,12,ppr


In [28]:
seasons = list(np.arange(2007, 2021))
teams = [10, 12]
scoring_systems = ['standard', 'half-ppr', 'ppr']

dfs = []

for scoring_system in scoring_systems:
    # print(f"Working on scoring system: {scoring_system}")
    for team in teams:
        # print(f"Working on team: {team}")
        for season in seasons:
            # print(f"Working on season: {season}")
            dfs.append(fantasy_calculator_adp(scoring_system, team, season))

df_all = pd.concat(dfs)
df_all


No data available for combination: half-ppr, 10, 2007
No data available for combination: half-ppr, 10, 2008
No data available for combination: half-ppr, 10, 2009
No data available for combination: half-ppr, 10, 2010
No data available for combination: half-ppr, 10, 2011
No data available for combination: half-ppr, 10, 2012
No data available for combination: half-ppr, 10, 2013
No data available for combination: half-ppr, 10, 2014
No data available for combination: half-ppr, 10, 2015
No data available for combination: half-ppr, 10, 2016
No data available for combination: half-ppr, 10, 2017
No data available for combination: half-ppr, 12, 2007
No data available for combination: half-ppr, 12, 2008
No data available for combination: half-ppr, 12, 2009
No data available for combination: half-ppr, 12, 2010
No data available for combination: half-ppr, 12, 2011
No data available for combination: half-ppr, 12, 2012
No data available for combination: half-ppr, 12, 2013
No data available for combin

,season,teams,scoring,player_id,name,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye
0,2008,10,standard,1173.0,LaDainian Tomlinson,RB,LAC,1.4,1.01,813.0,1.0,5.0,0.7,8.0
1,2008,10,standard,925.0,Adrian Peterson,RB,MIN,2.4,1.02,417.0,1.0,6.0,0.7,7.0
2,2008,10,standard,1237.0,Brian Westbrook,RB,PHI,2.8,1.03,405.0,1.0,9.0,0.7,7.0
3,2008,10,standard,4.0,Joseph Addai,RB,IND,4.7,1.05,559.0,1.0,8.0,1.1,14.0
4,2008,10,standard,581.0,Steven Jackson,RB,LAR,5.2,1.05,658.0,1.0,11.0,1.4,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2020,12,ppr,2340.0,Chris Boswell,PK,PIT,170.4,15.02,49.0,132.0,180.0,14.5,9.0
199,2020,12,ppr,2531.0,Kaimi Fairbairn,PK,HOU,172.5,15.04,135.0,160.0,182.0,4.8,6.0
200,2020,12,ppr,2529.0,Jake Elliott,PK,PHI,173.1,15.05,122.0,164.0,183.0,4.2,7.0
201,2020,12,ppr,2336.0,Brandon McManus,PK,DEN,173.8,15.06,65.0,160.0,180.0,5.0,9.0


In [31]:
# Save to database
df_all.to_sql('adp', conn, index=False, if_exists='replace')

10330